In [2]:
from lib.utils import *
from lib.models import *
from lib.ekyn import *
from lib.env import *
from lib.datasets import *

In [3]:
trainloader = DataLoader(dataset=ShuffleSplitDataset(),batch_size=32,shuffle=True)
devloader = DataLoader(dataset=ShuffleSplitDataset(training=False),batch_size=32,shuffle=True)

In [4]:
class MODEL(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.fc1 = nn.Linear(45000,3)
    def forward(self,x):
        return self.fc1(x)
model = MODEL().to(DEVICE)

In [5]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=3e-4)
params = sum([p.flatten().size()[0] for p in list(model.parameters())])
print("Params: ",params)

Params:  135003


In [6]:
optimization_loop(model,trainloader,devloader,criterion,optimizer,epochs=20,DEVICE=DEVICE)

 Train Loss: 0.8617 Dev Loss: 0.8618:  20%|██        | 4/20 [02:56<11:47, 44.22s/it]


KeyboardInterrupt: 

In [ ]:
loss,metrics,y_true,y_pred,logits = evaluate(dataloader,model,criterion,DEVICE)
cm_grid(y_true=y_true,y_pred=y_pred)
print(metrics) 
print(loss)

In [ ]:
with torch.no_grad():
    y_true = torch.Tensor()
    y_pred = torch.Tensor()
    y_logits = torch.Tensor()
    loss_total = 0
    for (Xi,yi) in devloader:
        y_true = torch.cat([y_true,yi.argmax(axis=1)])

        Xi,yi = Xi.to(DEVICE),yi.to(DEVICE)
        logits = model(Xi,classification=False)
        # loss = criterion(logits,yi)
        # loss_total += loss.item()
        
        y_logits = torch.cat([y_logits,torch.softmax(logits,dim=1).detach().cpu()])
        # y_pred = torch.cat([y_pred,torch.softmax(logits,dim=1).argmax(axis=1).detach().cpu()])


In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
# logits_tsne = tsne.fit_transform(X_dev)
logits_tsne = tsne.fit_transform(y_logits.detach().cpu())
df = pd.DataFrame(logits_tsne)
df['y'] = y_true
sns.scatterplot(data=df,x=0,y=1,hue='y')